# Hand Gesture Recognition using LEAP GestRecog Dataset
This notebook loads, preprocesses, and trains a CNN model on the LEAP hand gesture recognition dataset.

In [2]:
# Install Kaggle API if not installed
!pip install -q kaggle

# Setup Kaggle API credentials
import os
if not os.path.exists("/root/.kaggle/kaggle.json"):
    from google.colab import files
    print("Upload your kaggle.json file (you can get it from your Kaggle account)")
    files.upload()  # Upload kaggle.json here manually

# Move kaggle.json to the right location and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset directly from Kaggle
!kaggle datasets download -d gti-upm/leapgestrecog --force

# Unzip the downloaded dataset
!unzip -q leapgestrecog.zip -d /content/leapgestrecog

print("Dataset downloaded and extracted to /content/leapgestrecog")


Upload your kaggle.json file (you can get it from your Kaggle account)


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
 99% 2.12G/2.13G [00:25<00:00, 49.4MB/s]
100% 2.13G/2.13G [00:25<00:00, 89.3MB/s]
Dataset downloaded and extracted to /content/leapgestrecog


In [3]:
# Install required packages
!pip install -q tensorflow opencv-python

In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

In [5]:
# Configuration
IMG_SIZE = 64
DATA_DIR = "/content/leapgestrecog/leapGestRecog"  # Make sure this path matches your uploaded folder
X, y = [], []
label_map = {}
label_index = 0

def load_images_from_folder(folder_path, label):
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label)

# Walk through dataset
for subject_folder in sorted(os.listdir(DATA_DIR)):
    subject_path = os.path.join(DATA_DIR, subject_folder)
    if os.path.isdir(subject_path):
        for gesture_folder in os.listdir(subject_path):
            gesture_path = os.path.join(subject_path, gesture_folder)
            if os.path.isdir(gesture_path) and gesture_folder.startswith("gesture"):
                gesture_num = int(gesture_folder.replace("gesture", ""))
                if gesture_num not in label_map:
                    label_map[gesture_num] = label_index
                    label_index += 1
                load_images_from_folder(gesture_path, label_map[gesture_num])

print("Number of images loaded:", len(X))
print("Label map:", label_map)

Number of images loaded: 0
Label map: {}


In [6]:
import os
import cv2

DATA_DIR = "/content/leapgestrecog/leapGestRecog"

X = []
y = []
IMG_SIZE = 64

# Check classes folders inside DATA_DIR
classes = os.listdir(DATA_DIR)
print("Classes found:", classes)

for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_dir):
        continue
    print(f"Loading class '{class_name}'")
    for file_name in os.listdir(class_dir):
        if file_name.endswith(".png") or file_name.endswith(".jpg"):
            img_path = os.path.join(class_dir, file_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label_idx)

print(f"Loaded {len(X)} images and {len(y)} labels")


Classes found: ['01', '06', '03', '04', '05', '02', '07', '00', '08', '09']
Loading class '01'
Loading class '06'
Loading class '03'
Loading class '04'
Loading class '05'
Loading class '02'
Loading class '07'
Loading class '00'
Loading class '08'
Loading class '09'
Loaded 0 images and 0 labels


In [7]:
for file_name in os.listdir(class_dir):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        img_path = os.path.join(class_dir, file_name)
        # rest of the code


In [8]:
for root, dirs, files in os.walk(class_dir):
    for file_name in files:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(root, file_name)
            # rest of the code


In [9]:
import os
import cv2

DATA_DIR = "/content/leapgestrecog/leapGestRecog"

X = []
y = []
IMG_SIZE = 64

classes = os.listdir(DATA_DIR)
print("Classes found:", classes)

for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_dir):
        continue
    print(f"Loading class '{class_name}'")
    print(f"Files in {class_dir}: {os.listdir(class_dir)}")  # Debug print

    for root, dirs, files in os.walk(class_dir):
        for file_name in files:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                img_path = os.path.join(root, file_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Failed to load image: {img_path}")
                    continue
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(label_idx)

print(f"Loaded {len(X)} images and {len(y)} labels")


Classes found: ['01', '06', '03', '04', '05', '02', '07', '00', '08', '09']
Loading class '01'
Files in /content/leapgestrecog/leapGestRecog/01: ['05_thumb', '07_ok', '06_index', '09_c', '01_palm', '03_fist', '04_fist_moved', '10_down', '02_l', '08_palm_moved']
Loading class '06'
Files in /content/leapgestrecog/leapGestRecog/06: ['05_thumb', '07_ok', '06_index', '09_c', '01_palm', '03_fist', '04_fist_moved', '10_down', '02_l', '08_palm_moved']
Loading class '03'
Files in /content/leapgestrecog/leapGestRecog/03: ['05_thumb', '07_ok', '06_index', '09_c', '01_palm', '03_fist', '04_fist_moved', '10_down', '02_l', '08_palm_moved']
Loading class '04'
Files in /content/leapgestrecog/leapGestRecog/04: ['05_thumb', '07_ok', '06_index', '09_c', '01_palm', '03_fist', '04_fist_moved', '10_down', '02_l', '08_palm_moved']
Loading class '05'
Files in /content/leapgestrecog/leapGestRecog/05: ['05_thumb', '07_ok', '06_index', '09_c', '01_palm', '03_fist', '04_fist_moved', '10_down', '02_l', '08_palm_mo

In [10]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Convert lists to arrays and normalize pixel values to [0,1]
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1).astype("float32") / 255.0

# One-hot encode labels
y = to_categorical(y, num_classes=len(classes))

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


Train shape: (16000, 64, 64, 1) (16000, 10)
Test shape: (4000, 64, 64, 1) (4000, 10)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,914 (2.61 MB)

 Trainable params: 683,914 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.1,
    verbose=2
)


Epoch 1/15
225/225 - 8s - 37ms/step - accuracy: 0.6885 - loss: 0.8819 - val_accuracy: 0.9850 - val_loss: 0.0714
Epoch 2/15
225/225 - 2s - 8ms/step - accuracy: 0.9541 - loss: 0.1261 - val_accuracy: 0.9875 - val_loss: 0.0248
Epoch 3/15
225/225 - 2s - 8ms/step - accuracy: 0.9745 - loss: 0.0697 - val_accuracy: 0.9906 - val_loss: 0.0147
Epoch 4/15
225/225 - 2s - 8ms/step - accuracy: 0.9801 - loss: 0.0495 - val_accuracy: 0.9919 - val_loss: 0.0141
Epoch 5/15
225/225 - 3s - 11ms/step - accuracy: 0.9844 - loss: 0.0335 - val_accuracy: 0.9906 - val_loss: 0.0124
Epoch 6/15
225/225 - 3s - 12ms/step - accuracy: 0.9835 - loss: 0.0345 - val_accuracy: 0.9894 - val_loss: 0.0206
Epoch 7/15
225/225 - 2s - 11ms/step - accuracy: 0.9844 - loss: 0.0336 - val_accuracy: 0.9912 - val_loss: 0.0117
Epoch 8/15
225/225 - 2s - 8ms/step - accuracy: 0.9845 - loss: 0.0316 - val_accuracy: 0.9906 - val_loss: 0.0124
Epoch 9/15
225/225 - 2s - 8ms/step - accuracy: 0.9867 - loss: 0.0263 - val_accuracy: 0.9912 - val_loss: 0.01

In [13]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc*100:.2f}%")


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9902 - loss: 0.0149
Test accuracy: 99.12%
